## Context-Enriched Retrieval in RAG
Retrieval-Augmented Generation (RAG) enhances AI responses by retrieving relevant knowledge from external sources. Traditional retrieval methods return isolated text chunks, which can lead to incomplete answers.

To address this, we introduce Context-Enriched Retrieval, which ensures that retrieved information includes neighboring chunks for better coherence.

Steps in This Notebook:
- Data Ingestion: Extract text from a PDF.
- Chunking with Overlapping Context: Split text into overlapping chunks to preserve context.
- Embedding Creation: Convert text chunks into numerical representations.
- Context-Aware Retrieval: Retrieve relevant chunks along with their neighbors for better completeness.
- Response Generation: Use a language model to generate responses based on retrieved context.
- Evaluation: Assess the model's response accuracy.

### RAG中的上下文增强检索  
检索增强生成（RAG）通过从外部资源中检索相关知识来提升AI响应能力。传统检索方法返回孤立的文本块，可能导致答案不完整。  

为解决这一问题，我们引入**上下文增强检索**，确保检索信息包含相邻文本块，以提升连贯性。  


### 本笔记本的步骤：  
1. **数据摄入**：从PDF中提取文本。  
2. **带重叠上下文的分块**：将文本切分为重叠的片段，保留上下文信息。  
3. **嵌入创建**：将文本块转换为数值化的向量表示。  
4. **上下文感知检索**：检索相关文本块时，同时获取其相邻片段以完善信息。  
5. **响应生成**：使用语言模型基于检索到的上下文生成回答。  
6. **评估**：评估模型响应的准确性。

## Setting Up the Environment
We begin by importing necessary libraries.

In [1]:
pip install PymuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 26.6 MB/s eta 0:00:00


In [2]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI

## Extracting Text from a PDF File
To implement RAG, we first need a source of textual data. In this case, we extract text from a PDF file using the PyMuPDF library.

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

## Chunking the Extracted Text
Once we have the extracted text, we divide it into smaller, overlapping chunks to improve retrieval accuracy.

In [4]:
def chunk_text(text, n, overlap):
    """
    Chunks the given text into segments of n characters with overlap.

    Args:
    text (str): The text to be chunked.
    n (int): The number of characters in each chunk.
    overlap (int): The number of overlapping characters between chunks.

    Returns:
    List[str]: A list of text chunks.
    """
    chunks = []  # Initialize an empty list to store the chunks

    # Loop through the text with a step size of (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Append a chunk of text from index i to i + n to the chunks list
        chunks.append(text[i:i + n])

    return chunks  # Return the list of text chunks

### 文本分块函数 `chunk_text` 详解

#### 函数功能与核心逻辑
这个函数的作用是将长文本分割成固定长度的片段（chunks），并通过重叠（overlap）机制保留上下文信息。其核心逻辑可以拆解为：  
1. 以 `n - overlap` 为步长遍历文本  
2. 每次提取从索引 `i` 开始、长度为 `n` 的子字符串  
3. 将所有子字符串存入列表并返回  

这种分块方式的关键在于 **重叠参数 `overlap`**，它确保相邻块之间有部分重复内容，避免文本语义被分块边界截断。


#### 代码逐行解析
```python
def chunk_text(text, n, overlap):
    """
    将文本分割为带重叠的固定长度块
    
    参数:
    text (str): 待分块的原始文本
    n (int): 每个块的字符长度
    overlap (int): 相邻块的重叠字符数
    
    返回:
    List[str]: 分块后的文本片段列表
    """
    chunks = []  # 初始化空列表存储分块结果
    
    # 遍历文本，步长为 (n - overlap)
    for i in range(0, len(text), n - overlap):
        # 提取从索引i开始、长度为n的文本块
        # 注意：i + n 可能超过文本长度，Python切片会自动处理边界
        chunk = text[i:i + n]
        chunks.append(chunk)  # 将块添加到结果列表
    
    return chunks  # 返回分块列表
```


#### 示例演示：分块过程可视化
假设输入文本为 `"ABCDEFGHIJKL"`（12个字符），设置 `n=5`，`overlap=2`：  
1. **步长** = `n - overlap` = 5 - 2 = 3  
2. **分块过程**：  
   - `i=0`：提取 `0:5` → `"ABCDE"`  
   - `i=3`：提取 `3:8` → `"DEFGH"`（与前一块重叠 `"DE"`）  
   - `i=6`：提取 `6:11` → `"GHIJK"`（与前一块重叠 `"GH"`）  
   - `i=9`：提取 `9:14` → `"KLM"`（实际文本长度不足时取剩余部分）  
3. **最终结果**：`["ABCDE", "DEFGH", "GHIJK", "KLM"]`  


#### 参数影响与最佳实践
| 参数       | 作用                                                                 | 推荐取值范围       | 典型问题示例                     |
|------------|----------------------------------------------------------------------|--------------------|----------------------------------|
| `n`（块长度） | 决定单个块的语义完整度，影响检索粒度                                 | 500-2000 字符       | `n`过小：语义碎片化（如句子断开）<br>`n`过大：检索精度下降 |
| `overlap`（重叠量） | 控制相邻块的上下文重叠程度，提升语义连贯性                           | `n*10%~n*30%`       | `overlap`过小：块间语义断裂<br>`overlap`过大：冗余信息增加 |

**最佳实践**：  
- 对于中文文本，建议 `n=1000-1500` 字符（约500-750个汉字），`overlap=200-300` 字符  
- 若文本包含大量专业术语或长句，可适当增大 `overlap`（如 `n*30%`）  


#### 应用场景与扩展方向
1. **典型应用**：  
   - **文档检索系统**：将书籍、论文分割为带上下文的块，提升检索相关性  
   - **LLM上下文处理**：将长文本分块后送入模型，解决token长度限制问题（如GPT-4支持8k-32k token）  

2. **功能扩展**：  
   - **智能分块**：基于句号、段落等语义边界分块，而非固定字符数  
   - **动态重叠**：根据文本复杂度自动调整 `overlap`（如技术文档设更大重叠）  

3. **性能优化**：  
   - 对于超长篇文本（如10万+字符），可使用生成器（generator）替代列表存储，减少内存占用  


#### 与传统分块的对比
| 分块方式         | 优点                     | 缺点                     | 适用场景                 |
|------------------|--------------------------|--------------------------|--------------------------|
| **固定长度重叠分块** | 实现简单、上下文连贯性好   | 可能截断句子             | 通用RAG系统、LLM上下文处理 |
| **语义边界分块**   | 块语义完整度高           | 实现复杂、计算成本高     | 专业文档处理             |
| **无重叠分块**     | 存储效率高、计算速度快   | 上下文断裂风险高         | 简单检索、轻量级应用     |

该函数通过平衡实现复杂度和上下文保留效果，成为RAG系统中最常用的分块方式之一。

## Setting Up the OpenAI API Client
We initialize the OpenAI client to generate embeddings and responses.

In [6]:
import os
from openai import OpenAI

# 直接设置API密钥（注意：生产环境不建议硬编码）
os.environ["OPENAI_API_KEY"] = "sk-xxxx9"

# 初始化客户端
client = OpenAI(
    base_url="http://xxxxx98:9000/v1/",
    api_key=os.getenv("OPENAI_API_KEY")
)

## Extracting and Chunking Text from a PDF File
Now, we load the PDF, extract text, and split it into chunks.

In [7]:
# Define the path to the PDF file
pdf_path = "AI_Information.pdf"

# Extract text from the PDF file
extracted_text = extract_text_from_pdf(pdf_path)

# Chunk the extracted text into segments of 1000 characters with an overlap of 200 characters
text_chunks = chunk_text(extracted_text, 1000, 200)

# Print the number of text chunks created
print("Number of text chunks:", len(text_chunks))

# Print the first text chunk
print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

## Creating Embeddings for Text Chunks
Embeddings transform text into numerical vectors, which allow for efficient similarity search.

In [8]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    Creates embeddings for the given text using the specified OpenAI model.

    Args:
    text (str): The input text for which embeddings are to be created.
    model (str): The model to be used for creating embeddings. Default is "BAAI/bge-en-icl".

    Returns:
    dict: The response from the OpenAI API containing the embeddings.
    """
    # Create embeddings for the input text using the specified model
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # Return the response containing the embeddings

# Create embeddings for the text chunks
response = create_embeddings(text_chunks)

InternalServerError: Error code: 503 - {'error': {'message': '当前分组 default 下对于模型 BAAI/bge-en-icl 无可用渠道 (request id: 202506141823341838460265YFHbJgQ)', 'type': 'new_api_error'}}

In [9]:
def create_embeddings(text, model="text-embedding-ada-002"):
    """
    使用OpenAI模型为输入文本创建嵌入向量

    参数:
    text (str/list): 输入文本（支持字符串或字符串列表）
    model (str): 嵌入模型名称，默认使用text-embedding-ada-002

    返回:
    dict: OpenAI API返回的嵌入向量响应
    """
    # 确保输入是列表格式（OpenAI嵌入API要求）
    if isinstance(text, str):
        text = [text]

    try:
        response = client.embeddings.create(
            model=model,
            input=text
        )
        return response
    except Exception as e:
        print(f"嵌入生成错误: {e}")
        return None

In [10]:
response = create_embeddings(text_chunks)

In [11]:
print(text_chunks)

['Understanding Artificial Intelligence \nChapter 1: Introduction to Artificial Intelligence \nArtificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot \nto perform tasks commonly associated with intelligent beings. The term is frequently applied to \nthe project of developing systems endowed with the intellectual processes characteristic of \nhumans, such as the ability to reason, discover meaning, generalize, or learn from past \nexperience. Over the past few decades, advancements in computing power and data availability \nhave significantly accelerated the development and deployment of AI. \nHistorical Context \nThe idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. \nHowever, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop \nin 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving \nand symbolic methods. The 1980s saw

In [12]:
print(response)

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.005045936442911625, -0.01235856395214796, -0.0002783621894195676, -0.016287071630358696, -0.011097876355051994, 0.011760056018829346, -0.008913956582546234, 0.02704749070107937, -0.006526289973407984, -0.014937244355678558, 0.014695294201374054, 0.02148263528943062, -0.019394222646951675, -0.0033140818122774363, -0.007551394868642092, 0.005326088983565569, 0.028983093798160553, 0.005447064060717821, -0.008818450383841991, -0.0031740053091198206, -0.029696209356188774, 0.01254321075975895, 0.0030562137253582478, -0.004835821688175201, -0.012434969656169415, -0.011301623657345772, 0.02290886826813221, -0.047651465982198715, -0.0003941640316043049, -0.030918695032596588, 0.009614339098334312, -0.007309444714337587, -0.016172464936971664, -0.007309444714337587, -0.014657091349363327, -0.007646901533007622, 0.003511462127789855, -0.017114797607064247, 0.014007646590471268, -0.007672369945794344, 0.03468802571296692, 0.023965809494256973,

## Implementing Context-Aware Semantic Search
We modify retrieval to include neighboring chunks for better context.

In [13]:
def cosine_similarity(vec1, vec2):
    """
    Calculates the cosine similarity between two vectors.

    Args:
    vec1 (np.ndarray): The first vector.
    vec2 (np.ndarray): The second vector.

    Returns:
    float: The cosine similarity between the two vectors.
    """
    # Compute the dot product of the two vectors and divide by the product of their norms
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [14]:
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    Retrieves the most relevant chunk along with its neighboring chunks.

    Args:
    query (str): Search query.
    text_chunks (List[str]): List of text chunks.
    embeddings (List[dict]): List of chunk embeddings.
    k (int): Number of relevant chunks to retrieve.
    context_size (int): Number of neighboring chunks to include.

    Returns:
    List[str]: Relevant text chunks with contextual information.
    """
    # Convert the query into an embedding vector
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []

    # Compute similarity scores between query and each text chunk embedding
    for i, chunk_embedding in enumerate(embeddings):
        # Calculate cosine similarity between the query embedding and current chunk embedding
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        # Store the index and similarity score as a tuple
        similarity_scores.append((i, similarity_score))

    # Sort chunks by similarity score in descending order (highest similarity first)
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # Get the index of the most relevant chunk
    top_index = similarity_scores[0][0]

    # Define the range for context inclusion
    # Ensure we don't go below 0 or beyond the length of text_chunks
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)

    # Return the relevant chunk along with its neighboring context chunks
    return [text_chunks[i] for i in range(start, end)]

### 上下文增强检索函数 `context_enriched_search` 详解

#### 函数核心功能与设计理念
该函数实现了RAG系统中的关键创新——**上下文增强检索**，其核心逻辑是：  
1. 找到与查询最相关的文本块  
2. 以该块为中心，向前后扩展`context_size`个相邻块  
3. 返回包含上下文的块集合，解决传统RAG中孤立块导致的语义断裂问题  

设计理念在于：**单一文本块可能无法完整表达语义，而相邻块的上下文能提供更完整的信息背景**，例如：  
- 前一句可能是后一句的前提（如“因为...所以...”）  
- 后一段可能是前一段的解释（如术语定义后的展开说明）


#### 代码逐模块解析
```python
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    检索相关文本块及其上下文
    
    参数:
    query (str): 用户查询
    text_chunks (List[str]): 文本块列表
    embeddings (List[dict]): 文本块的嵌入向量列表
    k (int): 检索的相关块数量（当前仅支持k=1）
    context_size (int): 扩展的相邻块数量
    
    返回:
    List[str]: 包含上下文的文本块列表
    """
    # 1. 生成查询的嵌入向量
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []
    
    # 2. 计算查询与所有文本块的相似度
    for i, chunk_embedding in enumerate(embeddings):
        # 将向量转换为numpy数组计算余弦相似度
        sim = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, sim))  # 存储(块索引, 相似度)元组
    
    # 3. 按相似度降序排序
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    
    # 4. 获取最相关块的索引
    top_index = similarity_scores[0][0]
    
    # 5. 确定上下文范围（防止索引越界）
    start = max(0, top_index - context_size)    # 前扩context_size个块
    end = min(len(text_chunks), top_index + context_size + 1)  # 后扩context_size个块（+1是因为Python切片左闭右开）
    
    # 6. 返回包含上下文的块列表
    return [text_chunks[i] for i in range(start, end)]
```


#### 关键步骤可视化
假设文本块索引为0-9，`context_size=1`，最相关块索引为4：  
1. **上下文范围计算**：  
   - `start = 4 - 1 = 3`（前扩1块）  
   - `end = 4 + 1 + 1 = 6`（后扩1块，切片到索引6不包含6）  
2. **返回块**：索引3、4、5，共3个块  
3. **可视化效果**：  
   ```
   [块3] ← 上下文1 | [块4] ← 最相关块 | [块5] ← 上下文2
   ```


#### 参数影响与调优策略
| 参数          | 作用                                                                 | 推荐取值 | 典型问题示例                     |
|---------------|----------------------------------------------------------------------|----------|----------------------------------|
| `context_size` | 控制上下文扩展的块数量，值越大上下文越完整，但可能引入无关信息       | 1-3      | 值=0：传统RAG（无上下文）<br>值>3：冗余信息增加 |
| `k`          | 预留的多相关块检索参数（当前未完全实现）                             | 1        | 未来可扩展为返回top-k个块的上下文 |

**调优策略**：  
- 简单问题：`context_size=1`（仅扩展前后各1块）  
- 复杂问题（如需要跨段落推理）：`context_size=2-3`  
- 结合文本块长度调整：若块长500字，`context_size=1`相当于扩展1000字上下文  


#### 与传统检索的对比
| 检索方式         | 核心差异                 | 优点                     | 缺点                     |
|------------------|--------------------------|--------------------------|--------------------------|
| **上下文增强检索** | 强制包含相邻块上下文     | 回答更连贯、减少信息断层 | 计算量略增加、可能冗余   |
| **传统检索**     | 仅返回最相关的单个块     | 计算效率高               | 回答可能碎片化、缺乏背景 |

**案例对比**：  
- 查询：“爱因斯坦的相对论主要结论是什么？”  
- 传统检索：返回包含“相对论结论”的块，但可能缺少“相对论前提假设”的前一块  
- 上下文增强检索：同时返回前一块（前提）+ 结论块 + 后一块（应用），回答更完整  


#### 性能优化与扩展方向
1. **大规模数据优化**：  
   - 用FAISS等向量数据库替代内存相似度计算，提升千万级文本块的检索效率  
   - 预处理时构建块索引，避免每次检索重新计算所有相似度  

2. **智能上下文调整**：  
   - 根据查询复杂度动态调整`context_size`（如复杂问题自动设为2，简单问题设为0）  
   - 引入注意力机制，对相邻块按相似度加权（而非固定全选）  

3. **多模态扩展**：  
   - 若文本块包含图像，可同时检索相邻块的图像嵌入，实现图文联合上下文检索  


#### 潜在风险与解决方案
1. **上下文冗余风险**：  
   - 当`context_size`过大时，可能包含与问题无关的块（如最相关块在文档开头，后扩块为其他主题）  
   - **解决方案**：在返回上下文后，用LLM先过滤无关块，再生成回答  

2. **边界块处理**：  
   - 最相关块在文档开头或结尾时，上下文扩展会缺失部分内容（如start=0时无前块）  
   - **解决方案**：在分块时预留文档首尾的额外上下文（如开头添加文档简介块，结尾添加总结块）  

3. **嵌入不一致问题**：  
   - 查询嵌入与文本块嵌入可能使用不同模型生成，导致相似度计算偏差  
   - **解决方案**：强制查询和文本块使用同一嵌入模型（如均用`text-embedding-ada-002`）  


#### 实际应用场景
1. **学术问答系统**：  
   - 检索论文中某公式的推导时，同时返回前后段落的理论背景和应用案例  

2. **法律文档检索**：  
   - 查找合同条款时，自动包含该条款的前提条件和后续责任说明块  

3. **客服知识库**：  
   - 回答用户问题时，不仅返回最相关政策条文，还包含其适用范围和例外情况的相邻块  

该函数通过“检索+上下文扩展”的轻量级设计，在不显著增加计算成本的前提下，大幅提升了RAG系统的回答质量，是现代语义检索的核心创新之一。

### 上下文增强检索函数 `context_enriched_search` 详解

#### 函数核心功能与设计理念
该函数实现了RAG系统中的关键创新——**上下文增强检索**，其核心逻辑是：  
1. 找到与查询最相关的文本块  
2. 以该块为中心，向前后扩展`context_size`个相邻块  
3. 返回包含上下文的块集合，解决传统RAG中孤立块导致的语义断裂问题  

设计理念在于：**单一文本块可能无法完整表达语义，而相邻块的上下文能提供更完整的信息背景**，例如：  
- 前一句可能是后一句的前提（如“因为...所以...”）  
- 后一段可能是前一段的解释（如术语定义后的展开说明）


#### 代码逐模块解析
```python
def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    """
    检索相关文本块及其上下文
    
    参数:
    query (str): 用户查询
    text_chunks (List[str]): 文本块列表
    embeddings (List[dict]): 文本块的嵌入向量列表
    k (int): 检索的相关块数量（当前仅支持k=1）
    context_size (int): 扩展的相邻块数量
    
    返回:
    List[str]: 包含上下文的文本块列表
    """
    # 1. 生成查询的嵌入向量
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []
    
    # 2. 计算查询与所有文本块的相似度
    for i, chunk_embedding in enumerate(embeddings):
        # 将向量转换为numpy数组计算余弦相似度
        sim = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, sim))  # 存储(块索引, 相似度)元组
    
    # 3. 按相似度降序排序
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    
    # 4. 获取最相关块的索引
    top_index = similarity_scores[0][0]
    
    # 5. 确定上下文范围（防止索引越界）
    start = max(0, top_index - context_size)    # 前扩context_size个块
    end = min(len(text_chunks), top_index + context_size + 1)  # 后扩context_size个块（+1是因为Python切片左闭右开）
    
    # 6. 返回包含上下文的块列表
    return [text_chunks[i] for i in range(start, end)]
```


#### 关键步骤可视化
假设文本块索引为0-9，`context_size=1`，最相关块索引为4：  
1. **上下文范围计算**：  
   - `start = 4 - 1 = 3`（前扩1块）  
   - `end = 4 + 1 + 1 = 6`（后扩1块，切片到索引6不包含6）  
2. **返回块**：索引3、4、5，共3个块  
3. **可视化效果**：  
   ```
   [块3] ← 上下文1 | [块4] ← 最相关块 | [块5] ← 上下文2
   ```


#### 参数影响与调优策略
| 参数          | 作用                                                                 | 推荐取值 | 典型问题示例                     |
|---------------|----------------------------------------------------------------------|----------|----------------------------------|
| `context_size` | 控制上下文扩展的块数量，值越大上下文越完整，但可能引入无关信息       | 1-3      | 值=0：传统RAG（无上下文）<br>值>3：冗余信息增加 |
| `k`          | 预留的多相关块检索参数（当前未完全实现）                             | 1        | 未来可扩展为返回top-k个块的上下文 |

**调优策略**：  
- 简单问题：`context_size=1`（仅扩展前后各1块）  
- 复杂问题（如需要跨段落推理）：`context_size=2-3`  
- 结合文本块长度调整：若块长500字，`context_size=1`相当于扩展1000字上下文  


#### 与传统检索的对比
| 检索方式         | 核心差异                 | 优点                     | 缺点                     |
|------------------|--------------------------|--------------------------|--------------------------|
| **上下文增强检索** | 强制包含相邻块上下文     | 回答更连贯、减少信息断层 | 计算量略增加、可能冗余   |
| **传统检索**     | 仅返回最相关的单个块     | 计算效率高               | 回答可能碎片化、缺乏背景 |

**案例对比**：  
- 查询：“爱因斯坦的相对论主要结论是什么？”  
- 传统检索：返回包含“相对论结论”的块，但可能缺少“相对论前提假设”的前一块  
- 上下文增强检索：同时返回前一块（前提）+ 结论块 + 后一块（应用），回答更完整  


#### 性能优化与扩展方向
1. **大规模数据优化**：  
   - 用FAISS等向量数据库替代内存相似度计算，提升千万级文本块的检索效率  
   - 预处理时构建块索引，避免每次检索重新计算所有相似度  

2. **智能上下文调整**：  
   - 根据查询复杂度动态调整`context_size`（如复杂问题自动设为2，简单问题设为0）  
   - 引入注意力机制，对相邻块按相似度加权（而非固定全选）  

3. **多模态扩展**：  
   - 若文本块包含图像，可同时检索相邻块的图像嵌入，实现图文联合上下文检索  


#### 潜在风险与解决方案
1. **上下文冗余风险**：  
   - 当`context_size`过大时，可能包含与问题无关的块（如最相关块在文档开头，后扩块为其他主题）  
   - **解决方案**：在返回上下文后，用LLM先过滤无关块，再生成回答  

2. **边界块处理**：  
   - 最相关块在文档开头或结尾时，上下文扩展会缺失部分内容（如start=0时无前块）  
   - **解决方案**：在分块时预留文档首尾的额外上下文（如开头添加文档简介块，结尾添加总结块）  

3. **嵌入不一致问题**：  
   - 查询嵌入与文本块嵌入可能使用不同模型生成，导致相似度计算偏差  
   - **解决方案**：强制查询和文本块使用同一嵌入模型（如均用`text-embedding-ada-002`）  


#### 实际应用场景
1. **学术问答系统**：  
   - 检索论文中某公式的推导时，同时返回前后段落的理论背景和应用案例  

2. **法律文档检索**：  
   - 查找合同条款时，自动包含该条款的前提条件和后续责任说明块  

3. **客服知识库**：  
   - 回答用户问题时，不仅返回最相关政策条文，还包含其适用范围和例外情况的相邻块  

该函数通过“检索+上下文扩展”的轻量级设计，在不显著增加计算成本的前提下，大幅提升了RAG系统的回答质量，是现代语义检索的核心创新之一。

## Running a Query with Context Retrieval
We now test the context-enriched retrieval.

In [15]:
# Load the validation dataset from a JSON file
with open('val.json') as f:
    data = json.load(f)

# Extract the first question from the dataset to use as our query
query = data[0]['question']

# Retrieve the most relevant chunk and its neighboring chunks for context
# Parameters:
# - query: The question we're searching for
# - text_chunks: Our text chunks extracted from the PDF
# - response.data: The embeddings of our text chunks
# - k=1: Return the top match
# - context_size=1: Include 1 chunk before and after the top match for context
top_chunks = context_enriched_search(query, text_chunks, response.data, k=1, context_size=1)

# Print the query for reference
print("Query:", query)
# Print each retrieved chunk with a heading and separator
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

Query: What is 'Explainable AI' and why is it considered important?
Context 1:
nt aligns with societal values. Education and awareness campaigns inform the public 
about AI, its impacts, and its potential. 
Chapter 19: AI and Ethics 
Principles of Ethical AI 
Ethical AI principles guide the development and deployment of AI systems to ensure they are fair, 
transparent, accountable, and beneficial to society. Key principles include respect for human 
rights, privacy, non-discrimination, and beneficence. 
 
 
Addressing Bias in AI 
AI systems can inherit and amplify biases present in the data they are trained on, leading to unfair 
or discriminatory outcomes. Addressing bias requires careful data collection, algorithm design, 
and ongoing monitoring and evaluation. 
Transparency and Explainability 
Transparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
f

In [16]:
print(top_chunks)

['nt aligns with societal values. Education and awareness campaigns inform the public \nabout AI, its impacts, and its potential. \nChapter 19: AI and Ethics \nPrinciples of Ethical AI \nEthical AI principles guide the development and deployment of AI systems to ensure they are fair, \ntransparent, accountable, and beneficial to society. Key principles include respect for human \nrights, privacy, non-discrimination, and beneficence. \n \n \nAddressing Bias in AI \nAI systems can inherit and amplify biases present in the data they are trained on, leading to unfair \nor discriminatory outcomes. Addressing bias requires careful data collection, algorithm design, \nand ongoing monitoring and evaluation. \nTransparency and Explainability \nTransparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) \ntechniques aim to make AI decisions more understandable, enabling users to assess their \nfairness and accuracy. \nPrivacy and Data Protection \nAI syst

### 验证数据集加载与上下文检索实现详解

#### 一、验证数据集加载流程
```python
# 从JSON文件加载验证数据集
with open('val.json') as f:
    data = json.load(f)

# 从数据集中提取第一个问题作为查询
query = data[0]['question']
```

#### 核心功能解析：
1. **数据格式要求**：
   - `val.json`需为标准JSON格式，包含一个问题列表
   - 每个问题对象需包含`question`字段（如`data[0]['question']`）
   - 典型数据结构示例：
     ```json
     [
       {
         "question": "人工智能的核心技术有哪些？",
         "ideal_answer": "人工智能的核心技术包括机器学习、自然语言处理...",
         "context": "...相关文本块..."
       },
       {
         "..."
       }
     ]
     ```

2. **数据用途**：
   - 验证集用于评估RAG系统的检索准确性和回答质量
   - `query`作为用户问题输入，驱动后续的上下文检索和回答生成


#### 二、上下文增强检索调用
```python
# 执行上下文增强检索
top_chunks = context_enriched_search(
    query=query,
    text_chunks=text_chunks,
    embeddings=response.data,
    k=1,
    context_size=1
)
```

#### 参数详解：
| 参数名          | 作用                                                                 | 取值说明                     |
|-----------------|----------------------------------------------------------------------|------------------------------|
| `query`         | 用户查询文本                                                         | 从验证集提取的问题字符串     |
| `text_chunks`   | 从PDF提取并分块的文本列表                                           | 由`chunk_text`函数生成        |
| `embeddings`    | 文本块的嵌入向量列表                                                 | 由`create_embeddings`生成     |
| `k`             | 检索的相关块数量                                                     | 目前仅支持k=1（取最相关块）  |
| `context_size`  | 扩展的相邻块数量                                                     | 1表示前后各扩展1个相邻块     |

#### 执行流程：
1. **查询嵌入生成**：`query`通过`create_embeddings`转换为1536维向量
2. **相似度计算**：遍历所有`embeddings`，计算与查询向量的余弦相似度
3. **上下文扩展**：找到最相关块后，向前后扩展`context_size`个块
4. **结果返回**：`top_chunks`包含`2*context_size+1`个块（如context_size=1时返回3个块）


#### 三、检索结果展示
```python
# 打印查询内容
print("Query:", query)

# 打印每个检索到的上下文块
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")
```

#### 输出格式说明：
- **查询展示**：先打印用户问题，便于对照检索结果
- **上下文块展示**：
  - 每个块添加`Context X:`编号
  - 块内容后添加`=====================================`分隔符
  - 典型输出示例：
    ```
    Query: 人工智能的核心技术有哪些？
    
    Context 1:
    人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，旨在创造能够模拟人类智能的系统。其发展依赖于算法、算力和数据三大要素...
    =====================================
    Context 2:
    人工智能的核心技术包括机器学习、自然语言处理（NLP）、计算机视觉、机器人学等。其中，机器学习是AI的基础，通过算法从数据中学习模式...
    =====================================
    Context 3:
    ...这些技术相互结合，推动了AI在医疗、金融、工业等领域的应用。例如，自然语言处理使AI能够理解和生成人类语言...
    =====================================
    ```


#### 四、关键技术点与注意事项
1. **验证集准备要点**：
   - `val.json`需包含与PDF内容相关的问题，确保检索有意义
   - 问题应覆盖PDF的不同主题，全面评估检索准确性
   - 建议包含50-100个测试样本，避免过拟合

2. **检索参数调优**：
   - `context_size`影响检索结果的上下文完整性：
     - 若返回块包含过多无关信息，可减小`context_size`（如设为0）
     - 若回答因上下文不足而不完整，可增大`context_size`（如设为2）
   - 可通过交叉验证确定最优`context_size`（如比较context_size=1和2时的评估得分）

3. **异常处理建议**：
   - 添加文件存在性检查：
     ```python
     if not os.path.exists('val.json'):
         raise FileNotFoundError("验证集文件val.json不存在，请检查路径")
     ```
   - 处理JSON解析错误：
     ```python
     try:
         with open('val.json') as f:
             data = json.load(f)
     except json.JSONDecodeError:
         print("错误：val.json格式不正确，请检查JSON语法")
         data = []
     ```
   - 确保数据结构有效性：
     ```python
     if not data or 'question' not in data[0]:
         raise ValueError("验证集格式错误，缺少question字段")
     ```


#### 五、扩展应用场景
1. **多轮对话场景**：
   - 可修改为加载历史对话作为上下文，实现多轮检索
   ```python
   # 假设历史对话存储为列表
   history = [{"role": "user", "content": "什么是AI？"}, {"role": "assistant", "content": "..."}]
   query = f"历史对话：{history}\n当前问题：{data[0]['question']}"
   ```

2. **批量评估场景**：
   - 遍历验证集中的所有问题，计算检索准确率
   ```python
   correct_count = 0
   for item in data:
       query = item['question']
       relevant_chunks = item['relevant_chunks']  # 预设的相关块索引
       retrieved_chunks = context_enriched_search(query, text_chunks, embeddings)
       # 检查检索结果是否包含预设相关块
       if any(i in retrieved_indices for i in relevant_chunks):
           correct_count += 1
   accuracy = correct_count / len(data)
   print(f"检索准确率：{accuracy:.2f}")
   ```

3. **可视化检索过程**：
   - 可添加相似度得分展示，辅助分析检索效果
   ```python
   # 修改context_enriched_search函数返回相似度得分
   def context_enriched_search(...):
       # ...原有逻辑...
       return [text_chunks[i] for i in range(start, end)], [scores[i][1] for i in range(start, end)]
   
   # 打印时展示得分
   top_chunks, scores = context_enriched_search(...)
   for i, (chunk, score) in enumerate(zip(top_chunks, scores)):
       print(f"Context {i+1} (相似度: {score:.4f}):\n{chunk}\n=====================================")
   ```


#### 六、常见问题与解决方案
1. **检索结果与问题无关**：
   - 原因1：PDF内容与验证集问题不匹配
     - 解决方案：确保PDF包含验证集问题的答案信息
   - 原因2：嵌入模型不适合当前文本
     - 解决方案：尝试更换嵌入模型（如`text-embedding-ada-002`→`BAAI/bge-large-en`）

2. **上下文块包含过多冗余信息**：
   - 解决方案：
     - 减小`context_size`（如从1改为0）
     - 添加冗余过滤逻辑：用LLM评估每个相邻块与查询的相关性，只保留相关度高的块

3. **JSON加载错误**：
   - 解决方案：
     - 使用`json.loads`前先检查文件内容格式
     - 采用容错性解析方式，跳过无效行
     ```python
     with open('val.json') as f:
         content = f.read()
         # 移除注释或无效字符（如有）
         data = json.loads(content)
     ```

通过上述代码和解析，实现了从验证集加载到上下文检索的完整流程，为RAG系统的准确性评估和优化提供了基础框架。

## Generating a Response Using Retrieved Context
We now generate a response using LLM.

In [18]:
# Define the system prompt for the AI assistant
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="gpt-4.1-mini"):
    """
    Generates a response from the AI model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The model to be used for generating the response. Default is "meta-llama/Llama-2-7B-chat-hf".

    Returns:
    dict: The response from the AI model.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)

In [19]:
print(ai_response)

ChatCompletion(id='chatcmpl-BiIOCQuuazG3D2xj22Yt216khCZjJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Explainable AI (XAI) refers to techniques that make AI decisions more understandable to users. It is considered important because it enables users to assess the fairness and accuracy of AI systems, thereby building trust by providing transparency and insights into the AI's decision-making processes.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1749896820, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_178c8d546f', usage=CompletionUsage(completion_tokens=54, prompt_tokens=652, total_t

In [20]:
print(ai_response.choices[0].message.content)

Explainable AI (XAI) refers to techniques that make AI decisions more understandable to users. It is considered important because it enables users to assess the fairness and accuracy of AI systems, thereby building trust by providing transparency and insights into the AI's decision-making processes.


### LLM响应生成模块深度解析：从上下文到答案的受控生成

#### 一、提示工程核心设计：system prompt的约束机制
```python
# 定义系统提示词，强制LLM基于上下文回答
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"
```

#### 设计原理与作用：
1. **行为约束**：
   - 通过`strictly answers based on the given context`强制LLM放弃内部知识，仅使用提供的上下文
   - 明确规定无上下文信息时的响应格式（`I do not have enough information...`），避免幻觉（hallucination）

2. **错误处理机制**：
   - 当上下文缺失关键信息时，引导LLM承认无知，而非编造答案
   - 这在医疗、法律等对准确性要求高的场景中至关重要

3. **与传统提示的区别**：
   | 类型             | 传统提示                          | 上下文约束提示                  |
   |------------------|-----------------------------------|---------------------------------|
   | **信息来源**     | 允许使用模型内部知识              | 仅允许使用提供的上下文          |
   | **回答风险**     | 可能产生幻觉                      | 幻觉风险极低                    |
   | **适用场景**     | 创意写作、闲聊                    | 专业问答、事实性检索            |


#### 二、响应生成函数：与LLM的交互接口
```python
def generate_response(system_prompt, user_message, model="gpt-4.1-mini"):
    """
    基于系统提示和用户消息生成LLM响应
    
    参数:
    system_prompt (str): 控制LLM行为的系统提示
    user_message (str): 用户问题+上下文组合的提示词
    model (str): 模型名称，默认使用轻量级的gpt-4.1-mini
    
    返回:
    dict: LLM响应对象，包含生成的文本内容
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response
```

#### 关键参数解析：
1. **model参数**：
   - `gpt-4.1-mini`是OpenAI的轻量级模型，具有以下特点：
     - 参数量级：约10亿（对比gpt-4的850亿）
     - 推理速度：比gpt-4快3-5倍
     - 成本：约为gpt-4的1/10
   - 适用场景：对响应速度要求高的实时问答、资源有限的中小型应用

2. **temperature=0**：
   - 温度参数控制输出随机性：
     - 0：完全确定性输出（相同输入必获相同回答）
     - 1：高随机性（适合创意场景）
   - 在此场景的必要性：
     - 确保评估时结果可复现
     - 避免随机性导致的回答质量波动

3. **messages结构**：
   - 遵循OpenAI聊天API格式，包含：
     - system角色：设定LLM行为基调
     - user角色：包含用户问题和上下文提示
   - 这种结构使LLM能理解自身角色（助手）和任务要求（基于上下文回答）


#### 三、提示词构建：上下文与问题的格式化拼接
```python
# 将检索到的上下文块格式化为带编号的提示词
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
# 添加用户问题到提示词末尾
user_prompt = f"{user_prompt}\nQuestion: {query}"
```

#### 格式设计的心理学原理：
1. **视觉分隔符**：
   - `Context X:`编号和`=====================================`分隔线创造视觉层次
   - 帮助LLM（以及人类开发者）快速区分不同上下文块

2. **信息排序策略**：
   - 先呈现上下文，再提出问题，符合"先事实后提问"的认知逻辑
   - 类似人类交流中"基于前文，我问你..."的表达方式

3. **认知负荷优化**：
   - 每个块单独标注，避免多块信息混合导致的理解困难
   - 实验表明，带分隔的上下文可使LLM回答准确率提升15-20%


#### 四、完整调用流程与数据流向
1. **输入数据链**：
   ```
   PDF文本 → 分块 → 检索上下文块 → 格式化提示词 → LLM输入
   ```

2. **调用时序图**：
   ```
   用户问题(query)
       ↓
   检索上下文(top_chunks)
       ↓
   格式化提示词(user_prompt)
       ↓
   system prompt + user_prompt → LLM → ai_response
   ```

3. **示例提示词生成**：
   **输入**：
   - query = "人工智能的核心技术有哪些？"
   - top_chunks = [块1（AI定义），块2（核心技术列表），块3（技术应用）]
   
   **生成的user_prompt**：
   ```
   Context 1:
   人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，旨在创造能够模拟人类智能的系统。其发展依赖于算法、算力和数据三大要素...
   =====================================
   Context 2:
   人工智能的核心技术包括机器学习、自然语言处理（NLP）、计算机视觉、机器人学等。其中，机器学习是AI的基础，通过算法从数据中学习模式...
   =====================================
   Context 3:
   ...这些技术相互结合，推动了AI在医疗、金融、工业等领域的应用。例如，自然语言处理使AI能够理解和生成人类语言...
   =====================================
   Question: 人工智能的核心技术有哪些？
   ```

   **LLM输出**：
   ```
   人工智能的核心技术包括机器学习、自然语言处理（NLP）、计算机视觉和机器人学等。其中，机器学习是AI的基础，通过算法从数据中学习模式...（内容来自块2，可能结合块1和3的补充信息）
   ```


#### 五、优化方向与高级技巧
1. **提示词长度控制**：
   - 超过模型token限制时（如gpt-4.1-mini支持8k token），需截断上下文
   - 优化策略：
     - 按相似度排序后只保留最相关的前N个块
     - 使用LLM先总结长上下文，再生成回答
   ```python
   # 示例：当提示词过长时自动总结
   if len(user_prompt) > 4000:
       summary_prompt = f"请总结以下上下文：{user_prompt}\n总结："
       summary = generate_response(system_prompt, summary_prompt)
       user_prompt = f"总结：{summary}\nQuestion: {query}"
   ```

2. **多轮上下文管理**：
   - 在对话场景中，保留历史消息作为上下文
   ```python
   # 假设历史消息存储为列表
   history = [
       {"role": "user", "content": "什么是AI？"},
       {"role": "assistant", "content": "AI是..."}
   ]
   user_prompt = f"历史对话：{history}\n{user_prompt}"
   ```

3. **模型切换策略**：
   - 根据问题复杂度动态选择模型：
     - 简单问题：使用gpt-4.1-mini（快速低成本）
     - 复杂问题：自动切换至gpt-4（高精度）
   ```python
   def select_model(question):
       # 简单逻辑：问题长度>50字则用gpt-4，否则用mini
       if len(question) > 50:
           return "gpt-4"
       return "gpt-4.1-mini"
   model = select_model(query)
   ai_response = generate_response(system_prompt, user_prompt, model)
   ```


#### 六、潜在风险与解决方案
1. **上下文不足风险**：
   - 当检索到的上下文不包含答案时，LLM会返回预设的无信息回答
   - 解决方案：
     - 提高检索相关性（优化`context_enriched_search`）
     - 添加 fallback 机制：若上下文无答案，允许LLM使用内部知识（但需明确标注）

2. **模型偏见风险**：
   - 不同模型可能对相同上下文生成不同回答
   - 解决方案：
     - 多模型投票（如同时调用gpt-4和Claude，取共识答案）
     - 对关键问题添加人工审核环节

3. **成本控制问题**：
   - gpt-4等高级模型成本较高
   - 解决方案：
     - 使用模型蒸馏（如从gpt-4蒸馏出轻量级模型）
     - 对非关键问题使用免费模型（如开源的Llama-2）

通过该模块的设计，实现了从检索上下文到生成准确回答的关键转换，其中提示工程和模型选择是影响回答质量的两大核心因素，需根据具体应用场景持续优化。

## Evaluating the AI Response
We compare the AI response with the expected answer and assign a score.

In [21]:
# Define the system prompt for the evaluation system
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# Create the evaluation prompt by combining the user query, AI response, true response, and evaluation system prompt
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Generate the evaluation response using the evaluation system prompt and evaluation prompt
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation response
print(evaluation_response.choices[0].message.content)

1


In [22]:
print(evaluation_response)

ChatCompletion(id='chatcmpl-BiIPEmFY9Gex4tvHMF8JXj20YhZ8D', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1749896884, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_178c8d546f', usage=CompletionUsage(completion_tokens=2, prompt_tokens=284, total_tokens=286, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_t

### 上下文增强RAG系统完整解析：从PDF处理到智能评估

#### 一、系统整体架构与核心目标
该Notebook实现了一个**上下文增强的检索增强生成（RAG）系统**，核心目标是通过引入相邻文本块的上下文信息，解决传统RAG中孤立文本块导致的答案不完整问题。系统全流程涵盖6大模块：数据摄入、带重叠的文本分块、嵌入生成、上下文感知检索、响应生成和评估，形成完整的AI问答闭环。


#### 二、数据处理模块：从PDF到文本分块

##### 1. PDF文本提取
```python
def extract_text_from_pdf(pdf_path):
    mypdf = fitz.open(pdf_path)
    all_text = ""
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")
        all_text += text
    return all_text
```
- **技术细节**：使用PyMuPDF（fitz）库解析PDF，逐页提取文本并拼接为完整字符串。该库支持复杂PDF格式，包括表格、图片旁文本等场景的文本识别。
- **应用场景**：适用于学术论文、技术手册、法律合同等PDF文档的批量文本提取。

##### 2. 带重叠的文本分块
```python
def chunk_text(text, n, overlap):
    chunks = []
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])
    return chunks
```
- **核心逻辑**：将长文本切分为固定长度（`n`）的片段，并通过`overlap`参数保留相邻块的重叠部分（如n=1000，overlap=200时，块间重叠200字符）。
- **设计优势**：
  - 避免关键信息被分块边界截断（如句子中途断开）。
  - 重叠上下文为后续检索提供更完整的语义信息，提升相关性。
- **参数影响**：
  - `n`过小会导致语义碎片化，过大则增加检索计算量；
  - `overlap`建议设为`n`的10%-30%，平衡上下文完整性和冗余度。


#### 三、向量检索模块：从文本到语义表示

##### 1. 文本嵌入生成
```python
def create_embeddings(text, model="text-embedding-ada-002"):
    if isinstance(text, str):
        text = [text]
    try:
        response = client.embeddings.create(model=model, input=text)
        return response
    except Exception as e:
        print(f"嵌入生成错误: {e}")
        return None
```
- **技术要点**：
  - 使用OpenAI的`text-embedding-ada-002`模型（1536维向量），将文本转换为稠密数值向量。
  - 输入支持字符串或列表，自动处理单文本场景（转换为列表）。
- **模型选择**：
  - 相比对话模型（如gpt-3.5-turbo），专用嵌入模型在语义表征精度和成本上更优（前者成本仅为后者的1/20）。

##### 2. 上下文感知检索
```python
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def context_enriched_search(query, text_chunks, embeddings, k=1, context_size=1):
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []
    for i, chunk_embedding in enumerate(embeddings):
        sim = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, sim))
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_index = similarity_scores[0][0]
    start = max(0, top_index - context_size)
    end = min(len(text_chunks), top_index + context_size + 1)
    return [text_chunks[i] for i in range(start, end)]
```
- **检索逻辑拆解**：
  1. **向量相似度计算**：使用余弦相似度衡量查询与文本块的语义距离（值越接近1越相似）。
  2. **上下文扩展**：找到最相关块后，向前后扩展`context_size`个相邻块（如context_size=1时，返回前1块+当前块+后1块，共3块）。
  3. **边界处理**：通过`max(0, ...)`和`min(len(...), ...)`避免索引越界。
- **关键参数**：
  - `context_size`：控制上下文扩展的范围，值越大上下文越完整，但可能引入无关信息。
  - `k`：预留的多结果检索参数，当前仅取top-1结果。


#### 四、LLM交互模块：从上下文到回答生成

##### 1. 响应生成函数
```python
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="gpt-4.1-mini"):
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response
```
- **提示工程设计**：
  - `system_prompt`强制LLM仅基于提供的上下文回答，避免幻觉（hallucination）。
  - `temperature=0`确保输出确定性，避免随机性影响结果一致性。
- **模型选择**：`gpt-4.1-mini`为轻量级模型，适合快速推理场景，性能与`gpt-3.5-turbo`接近但成本更低。

##### 2. 提示词构建
```python
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"
```
- **格式设计**：
  - 为每个上下文块添加编号和分隔符，提升LLM对多源信息的解析效率。
  - 将问题置于所有上下文之后，符合"先事实后提问"的人类认知逻辑。


#### 五、评估模块：量化回答质量

##### 1. 评估体系设计
```python
evaluate_system_prompt = "You are an intelligent evaluation system... assign a score of 0.5."

evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"
```
- **评分标准**：
  - 1.0：回答与真实答案高度一致；
  - 0.5：部分正确但存在遗漏或次要错误；
  - 0.0：完全错误或未回答问题。
- **评估逻辑**：通过LLM自身进行评估，本质是基于提示词的自动化人工模拟，需注意：
  - 评估模型需与生成模型独立（避免自评估偏差）；
  - 真实答案需来自权威源（如`data[0]['ideal_answer']`）。

##### 2. 典型输出分析
```python
# 假设评估输出为"0.5"
print(evaluation_response.choices[0].message.content)  # 输出：0.5
```
- **含义解读**：AI回答部分符合真实答案，可能存在以下情况：
  - 回答了问题的主要方面，但遗漏了关键细节；
  - 存在不影响核心的次要事实性错误；
  - 上下文提供的信息不完整，导致回答不充分。


#### 六、系统优化与扩展方向

##### 1. 性能优化点
- **嵌入缓存**：对相同文本块的嵌入结果进行缓存，避免重复计算（如使用Redis存储`text_hash:embedding`映射）。
- **向量数据库替换**：将内存中余弦相似度计算替换为FAISS/Chroma等专业向量数据库，提升大规模数据检索效率。

##### 2. 功能扩展建议
- **多模态支持**：增加图像嵌入模块（如CLIP模型），支持图文混合RAG。
- **动态上下文调整**：根据问题复杂度自动调整`context_size`（如复杂问题扩展更多上下文）。
- **长文档分层次检索**：先检索章节级别上下文，再细化到段落级，减少无关信息干扰。

##### 3. 潜在风险与对策
- **上下文冗余**：当`context_size`过大时，可能引入无关文本导致回答偏离，可通过TF-IDF过滤冗余块。
- **评估偏差**：LLM评估可能存在主观性，建议结合人工抽查（如10%样本）校准自动评估结果。


#### 七、完整流程总结
1. **文档理解阶段**：PDF→文本→带重叠的语义块，保留上下文完整性；
2. **语义映射阶段**：文本块→1536维向量，建立文本与数值的语义映射；
3. **智能检索阶段**：查询→最相关块→扩展上下文块，解决传统RAG的碎片化问题；
4. **受控生成阶段**：上下文+问题→LLM回答，通过system prompt约束回答边界；
5. **闭环评估阶段**：回答→与真实答案比对→量化评分，形成优化反馈链路。

该系统通过"上下文增强"这一核心创新，在传统RAG基础上提升了回答的连贯性和完整性，尤其适合处理需要跨段落理解的复杂问题场景。